In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
import uuid
import os
import streamlit as st  
import pandas as pd
import sqlite3
from dotenv import load_dotenv

### Load API key

In [2]:
# to load api key from .env file
load_dotenv()

OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

### Define LLM

In [3]:
llm = ChatOpenAI(model="gpt-4o-mini")

### Process PDF

This section reads in a order file, splits it into chunks, creates embeddings, builds a vectorstore, initializes a LLM and the respective prompt of it. Splitting documents into smaller Chunks is necessary since feeding the LLM with all of it could confuse it. Embedding are textual respresentation of words in a multi-dimensional vector space. The distances of each vector are in relation to the contect or meaning of words to each other. 

In [4]:
# load PDF
loader = PyPDFLoader("data/titanfreight_order.pdf")
pages = loader.load()

In [5]:
# split document due to token limits of llm's

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400,                 # max size for each chunk (in characters)
                                            chunk_overlap=60,                  # how many characters overlap betw. chunks
                                            length_function=len,               # to measure chunk length
                                            separators=["\n\n", "\n", " "])

chunks = text_splitter.split_documents(pages)

In [6]:
# create embeddings so llm can process data

def get_embedding_function():             
    embeddings = OpenAIEmbeddings(
        model="text-embedding-ada-002", openai_api_key=OPENAI_API_KEY
    )
    return embeddings

embedding_function = get_embedding_function()

In [7]:
# create vector database

def create_vectorstore(chunks, embedding_function, vectorstore_path):

    # Create a list of unique ids for each document based on the content (no duplicates)
    ids = [str(uuid.uuid5(uuid.NAMESPACE_DNS, doc.page_content)) for doc in chunks]
    
    # Ensure that only unique docs with unique ids are kept
    unique_ids = set()
    unique_chunks = []
    
    for chunk, id in zip(chunks, ids):     
        if id not in unique_ids:       
            unique_ids.add(id)
            unique_chunks.append(chunk) 

    # Create a new Chroma database from the documents
    vectorstore = Chroma.from_documents(documents=unique_chunks, 
                                        ids=list(unique_ids),
                                        embedding=embedding_function,
                                        persist_directory = vectorstore_path)
    
    return vectorstore

In [8]:
# Create vectorstore
vectorstore = create_vectorstore(chunks=chunks, 
                                 embedding_function=embedding_function,
                                 vectorstore_path="vectorstore_chroma")

In [9]:
# Load vectorstore
vectorstore = Chroma(persist_directory="vectorstore_chroma", embedding_function=embedding_function)

In [10]:
llm = ChatOpenAI(model="gpt-4o-mini")

retriever = vectorstore.as_retriever(search_type="similarity")

In [11]:
# Prompt template

PROMPT_TEMPLATE = """
You are an assistant for extracting information out of documents (transport orders).
Use the following pieces of retrieved context to answer
the question. 

{context}

---

Execute the following request: {question}

"""

### Generate responses

In this section, the LLM uses the vectorstore and the generated embeddings and to create a dataframe, organizing the data into predefined columns with the corresponding data from the documents.

In [12]:
# Create prompt

prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

In [13]:
# function to combine several documents into one and seperating them by two breaks (\n)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [14]:
# Generate structured responses

class AnswerWithSources(BaseModel):
    """An answer to the question, with sources and reasoning."""
    answer: str = Field(description="Answer to question")
    
class ExtractedInfo(BaseModel):
    """Extracted information about the research article"""
    Orderer: AnswerWithSources
    Loading_location: AnswerWithSources
    Unloading_location: AnswerWithSources
    Loading_date: AnswerWithSources
    Loading_time_window: AnswerWithSources
    Unloading_date: AnswerWithSources
    Unloading_time_window: AnswerWithSources
    Goods: AnswerWithSources
    Weight: AnswerWithSources
    ADR: AnswerWithSources
    Loading_number: AnswerWithSources
    Transport_rate: AnswerWithSources

In [15]:
rag_chain = (
            {"context": retriever | format_docs, "question": RunnablePassthrough()}
            | prompt_template
            | llm.with_structured_output(ExtractedInfo, strict=True)
        )

In [16]:
# Transform response into a dataframe 

structured_response = rag_chain.invoke("Give me the ordering company, full loading location, full unloading location, loading date, loading time window, unloading date, unloading time window, type of shipment goods, weight, indication of ADR, loading number, transport rate.")
df = pd.DataFrame([structured_response.dict()])

answer_row = []

for col in df.columns:
    answer_row.append(df[col][0]['answer'])

# Create new dataframe
df_orderlist = pd.DataFrame([answer_row], columns=df.columns, index=['answer'])
df_orderlist

Number of requested results 4 is greater than number of elements in index 3, updating n_results = 3


,Orderer,Loading_location,Unloading_location,Loading_date,Loading_time_window,Unloading_date,Unloading_time_window,Goods,Weight,ADR,Loading_number,Transport_rate
answer,Titan Freight Solutions,"Waldstraße 22, 44137 Dortmund","Hauptstraße 56, 60329 Frankfurt","November 20, 2024",09:00 AM to 11:00 AM,"November 20, 2024",03:00 PM to 05:00 PM,25 pallets of construction materials,"5,000 kilograms",No hazardous goods involved,Not specified in the provided context,"€2,000"


### Create SQL table and save order

This sets up a sqlite database and appends the data of each processed file to it.

In [17]:
# Create a SQLite connection
conn = sqlite3.connect("order_list.db")
cursor = conn.cursor()

# Creatomg a table with a primary key
cursor.execute("""
CREATE TABLE IF NOT EXISTS order_list (
    id_column INTEGER PRIMARY KEY AUTOINCREMENT,
    Orderer TEXT,
    Loading_location TEXT,
    Unloading_location TEXT,
    Loading_date TEXT,
    Loading_time_window TEXT,
    Unloading_date TEXT,
    Unloading_time_window TEXT,
    Goods TEXT,
    Weight TEXT,
    ADR TEXT,
    Loading_number TEXT,
    Transport_rate TEXT
);
""")
conn.commit()

# Open the SQLite connection
conn = sqlite3.connect("order_list.db")

# Save the DataFrame to the SQLite database
df_orderlist.to_sql("order_list", conn, if_exists="append", index=False)

conn.close()

### Query SQL table

Here, the database gets loaded and queried with a preset query which can be adjusted as desired.

In [18]:
# Connect to the database
conn = sqlite3.connect("order_list.db")

# Query to check if the table exists and view its contents
query = "SELECT * FROM order_list LIMIT 5;"  # Adjust LIMIT for more rows
result = pd.read_sql(query, conn)

# Close the connection
conn.close()

# Print the result
result

,id_column,Orderer,Loading_location,Unloading_location,Loading_date,Loading_time_window,Unloading_date,Unloading_time_window,Goods,Weight,ADR,Loading_number,Transport_rate
0,1,Titan Freight Solutions,"Waldstraße 22, 44137 Dortmund","Hauptstraße 56, 60329 Frankfurt","November 20, 2024",09:00 AM to 11:00 AM,"November 20, 2024",03:00 PM to 05:00 PM,25 pallets of construction materials,"5,000 kilograms",No hazardous goods involved,Not specified in the provided context,"€2,000"
